#**GitHub WebScraping Project**

##**Outline:**
####**This project has 2 phases.**
* **In phase 1 I have to scrape top 120 topic names ,their descriptions and their URLs from https://github.com/topics.**
* **In phase 2  for first 20 topic I have to scrape 100 top repos,repo username,repo title ,repo URLs and the no of stars of repo.**

##**Phase1:**
* **In this phase i will get a list of 100 topics.**
* **For each topic I will get topic title,topic descriptions, and topic page URL.**


##**Using the requests library to download web pages**
* I have to import requests library.
* Then i do get request to URL.
* In return i got all contents of URL webpage.

In [1]:
topic_url="https://github.com/topics?page=1"
import requests
response=requests.get(topic_url)
page_contents=response.text

###**Using BeautifulSoup to parse and extract information**
* Now i have to import BeautifulSoup library.
* After importing i parsed the content to the BeautifulSoup
* Then i extracted useful information from parsed content.



In [2]:
from bs4 import BeautifulSoup
doc=BeautifulSoup(page_contents,"lxml")

In [3]:
#Getting list of Topics 
topics=[]
topic_title_tags=doc.find_all("p",class_="f3 lh-condensed mb-0 mt-1 Link--primary")
print(len(topic_title_tags))
for tags in topic_title_tags:
  topics.append(tags.text)
# print(topics)

30


In [4]:
# Getting Descriptions of each topic
topic_descr=[]
topic_descr_tags=doc.find_all("p",class_="f5 color-fg-muted mb-0 mt-1")
for tags in topic_descr_tags:
  topic_descr.append(tags.text.strip())
# print(topic_descr)

In [ ]:
# Getting URL of the Topic page
topics_urls=[]
topic_url_tags=doc.find_all("a",class_="no-underline flex-grow-0")
for tags in topic_url_tags:
  url=(tags["href"])
  # we need to add baseurl to these topic url so
  base_url="https://github.com"
  topic_url=(base_url+url)
  topics_urls.append(topic_url)

###**Creating a csv file with the Extracted infromation**
* Now i have stored all of extracted information into a csv file.
* for this purpose i have acreated a dictionary and give it all information.
* After that i have imported pandas library to convert this dictionary into a dataframe and then store it in a csv file.

In [ ]:
topics_dict={
    "Topic": topics,
    "Topic Description":topic_descr ,
    "Topic URL": topics_urls ,
}
# now convert this dictionary into a DataFrame by using pandas library and export in csv
import pandas as pd 
topic_df=pd.DataFrame(topics_dict,index=[i for i in range (1,31)])
# now extracting dataframe into csv file
topic_df.to_csv("Topics info.csv")

###**Combining all code and making a single Function**
* Now i am creating a single function that will take starting and ending page and extract all information of that pages into a csv files.
* After creating a single function i will be able to scrape unlimited no of pages into a single csv file.



In [ ]:
def get_topic_info(startingpage,endingpage):
  all_df=[]
  for i in range (startingpage,endingpage+1):
    topic_url="https://github.com/topics?page={}".format(i)
    response=requests.get(topic_url)
    page_contents=response.text
    from bs4 import BeautifulSoup
    doc=BeautifulSoup(page_contents,"lxml")
    #Getting list of Topics 
    topics=[]
    topic_title_tags=doc.find_all("p",class_="f3 lh-condensed mb-0 mt-1 Link--primary")
    for tags in topic_title_tags:
      topics.append(tags.text)
    # Getting Descriptions of each topic
    topic_descr=[]
    topic_descr_tags=doc.find_all("p",class_="f5 color-fg-muted mb-0 mt-1")
    for tags in topic_descr_tags:
      topic_descr.append(tags.text.strip())
    # Getting URL of the Topic page
    topics_urls=[]
    topic_url_tags=doc.find_all("a",class_="no-underline flex-grow-0")
    for tags in topic_url_tags:
      url=(tags["href"])
      base_url="https://github.com"
      topic_url=(base_url+url)
      topics_urls.append(topic_url)
    # extracting all above informations into a csv file.
    # for this purpose first make a dictionary
    topics_dict={
        "Topic": topics,
        "Topic Description":topic_descr ,
        "Topic URL": topics_urls ,
    }
    # now convert this dictionary into a DataFrame by using pandas library and exporting in a csv file 
    import pandas as pd 
    topic_df=pd.DataFrame(topics_dict)
    all_df.append(topic_df)
    data_frame=pd.concat(all_df,ignore_index=True)
    # now extracting dataframe into csv file
    data_frame.to_csv("Topics info.csv")
  print("Data Scraped Succesfuly")

In [ ]:
# invoking function and getting data of page 1 to 4.you can give any range.
get_topic_info(1,4)

Data Scraped Succesfuly


##**Phase2**
* **In this phase I will get 100 repos of first 20 topics.**
* **For 100 repos of first 20 topics ,I will extract username,repo name,repoURL and no of stars of each repo.**
* **After extracting info I will store all info into a  csv for each topic.**

####first of all i get all info from first topic url and later i expanded it to all topic urls

In [ ]:
topic_url=topics_urls[0]
#  using request library
response=requests.get(topic_url)
content=response.text
# parsing content to BeautifulSoup
doc=BeautifulSoup(content,"lxml")
# getting username,reponame,repoURL and repo stars
h3_tags=doc.find_all("h3",class_="f3 color-fg-muted text-normal lh-condensed")
a_tags=h3_tags[0].find_all("a")
username=(a_tags[0].text.strip()) 
repo_name=(a_tags[1].text.strip()) 
repo_url=(base_url+a_tags[1]["href"])
star_tags=doc.find_all("span",class_="Counter js-social-count")
stars=(star_tags[0].text)
# creating a function that will no of stars in k and convert them into simple integer.
def parse_star_count(star_str):
  star_str=star_str.strip()
  if star_str[-1]=="k":
    no=int(float(star_str[:-1])*1000)
    return no  
  elif int(star_str) not in [i for i in range  (1,1000)]:
    return "Not Available"
  else: 
    return star_str
parse_star_count(stars)   

90000

 #### Now i defined a function that will take h3 tags and star tags from list of h3 tags and star tag and extract useful info from these tags.
#### This function will return user name,repo name,stars,repo URL

In [ ]:
def get_repo_info(h3_tags,star_tag):
  # return all the info about a reository
  a_tags=h3_tags.find_all("a")
  username=(a_tags[0].text.strip()) 
  repo_name=(a_tags[1].text.strip()) 
  repo_url=(base_url+a_tags[1]["href"])
  stars=parse_star_count(star_tag.text)
  return username,repo_name,stars,repo_url

#### Now i defined a function that will use previous function(get_repo_info) and create a dictionary of all information


In [ ]:
topic_repos_dict={
    "username":[],
    "repo_name":[],
    "stars":[],
    "repo_url":[]
}
for i in range (len(h3_tags)):
  repo_info=get_repo_info(h3_tags[i],star_tags[i])
  topic_repos_dict["username"].append(repo_info[0])
  topic_repos_dict["repo_name"].append(repo_info[1])
  topic_repos_dict["stars"].append(repo_info[2])
  topic_repos_dict["repo_url"].append(repo_info[3])

##**Combining all code into a single function**
* I created a function (get_topic_repos) that will take topic url and give all info of that repo in a dataframe.

In [ ]:
def get_topic_repos(topic_url):
  # download the page 
  response=requests.get(topic_url)
  # check the response
  if response.status_code!=200:
    raise Exception("Failed to load page{}".fromate(topic_url))
  page_content=response.text
  # parse using beautiful soup
  doc=BeautifulSoup(page_content,"lxml")
  # getting h3 tags containing username,reponame,repourl
  h3_tags=doc.find_all("h3",class_="f3 color-fg-muted text-normal lh-condensed")
  # finding startags containg no of stars
  star_tags=doc.find_all("span",class_="Counter js-social-count")
  # getting all of repo info
  topic_repos_dict={
    "Username":[],
    "Repo_name":[],
    "Stars":[],
    "Repo_url":[]}
  for i in range (len(h3_tags)):
    repo_info=get_repo_info(h3_tags[i],star_tags[i])
    topic_repos_dict["Username"].append(repo_info[0])
    topic_repos_dict["Repo_name"].append(repo_info[1])
    topic_repos_dict["Stars"].append(repo_info[2])
    topic_repos_dict["Repo_url"].append(repo_info[3])
  return pd.DataFrame(topic_repos_dict)

In [ ]:
# checking our get_topic_repo function
# get_topic_repos(topics_urls[10]).to_csv("atom.csv") #by giving any topic llink we can get its all repo information

# **Getting info of repos of every topic and storing them in separete csv files**

In [ ]:
# getting info of repos of every topic and storing them in separete csv fillles
file_no=0
data_frame=pd.read_csv("Topics info.csv")
for index,row in data_frame.iterrows():
  list_df=[]
  # print(row["Topic URL"])
  file_no+=1
  print("Sraping Topic no {} with title {}".format(file_no,row["Topic"]))
  for i in range(1,6):
    df=get_topic_repos(row["Topic URL"]+"?page={}".format(i))
    list_df.append(df)
  dm= pd.concat(list_df,ignore_index=True)
  dm.to_csv("{} Topicno {}.csv".format(row["Topic"],file_no))  
  if file_no==20:      # you can give no of your own choose where you want to stop
    break
print("{} Topics scraped Sucesfully".format(file_no))

Sraping Topic no 1 with title 3D
Sraping Topic no 2 with title Ajax
Sraping Topic no 3 with title Algorithm
Sraping Topic no 4 with title Amp
Sraping Topic no 5 with title Android
Sraping Topic no 6 with title Angular
Sraping Topic no 7 with title Ansible
Sraping Topic no 8 with title API
Sraping Topic no 9 with title Arduino
Sraping Topic no 10 with title ASP.NET
Sraping Topic no 11 with title Atom
Sraping Topic no 12 with title Awesome Lists
Sraping Topic no 13 with title Amazon Web Services
Sraping Topic no 14 with title Azure
Sraping Topic no 15 with title Babel
Sraping Topic no 16 with title Bash
Sraping Topic no 17 with title Bitcoin
Sraping Topic no 18 with title Bootstrap
Sraping Topic no 19 with title Bot
Sraping Topic no 20 with title C
20 Topics scraped Sucesfully


#**Conclusion:**
**This project was very interesting for me.I love to these web scarping project.I completed this project perfectly.**